## Проект: Анализ рынка недвижимости в Санкт Петербурге

### Описание проекта
В нашем распоряжении данные сервиса Яндекс Недвижимость — архив объявлений за несколько лет о продаже квартир в Санкт-Петербурге и соседних населённых пунктах.
#### Задача — выполнить предобработку данных и изучить их, чтобы найти интересные особенности и зависимости, которые существуют на рынке недвижимости.
О каждой квартире в базе содержится два типа данных: добавленные пользователем и картографические. Например, к первому типу относятся площадь квартиры, её этаж и количество балконов, ко второму — расстояния до центра города, аэропорта и ближайшего парка. 

#### План выполнения проекта
#### Шаг 1. Ознакомление с данными
* Изучение общей информации о полученном датафрейме.

#### Шаг 2. Предобработка данных
* Анализ датафрейма на пропуски и дубликаты.
* Заполнение пропущенных значений там, где это возможно.
* Преобразование типов данных.
* Анализ датафрейма на аномалии. 

#### Шаг 3. Добавление в таблицу новых столбцов со следующими параметрами:
* цена одного квадратного метра;
* день недели публикации объявления (0 — понедельник, 1 — вторник и так далее);
* месяц публикации объявления;
* год публикации объявления;
* тип этажа квартиры (значения — «первый», «последний», «другой»);
* расстояние до центра города в километрах (переведите из м в км и округлите до целых значений).

#### Шаг 4. Проведите исследовательский анализ данных:
* Изучение параметров объектов: общая площадь; жилая площадь; площадь кухни; цена объекта; количество комнат; высота потолков; этаж квартиры; тип этажа квартиры («первый», «последний», «другой»); общее количество этажей в доме; расстояние до центра города в метрах; расстояние до ближайшего аэропорта; расстояние до ближайшего парка; день и месяц публикации объявления.
* Построение гистограмм для каждого из этих параметров. 
* Выводы по полученным резульатам
* Изучение того, как быстро продавались квартиры.  
* Выводы согласно среднего времени, необходимого для продажи квартиры. Определение сроков быстрых и медленных продаж.
* Определение факторов, влияющих на общую (полную) стоимость объекта? Зависит ли цена от: общей площади; жилой площади; площади кухни; количества комнат; этажа, на котором расположена квартира (первый, последний, другой); даты размещения (день недели, месяц, год).
* Построение графиков, которые покажут зависимость цены от указанных выше параметров. 
* Определение средней цены одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений. Определение населённых пунктов с самой высокой и низкой стоимостью квадратного метра.
* Определение зависимости цены одного квадратного метра от расстояния от центра города

#### Шаг 5. Общий вывод на основании проведенного анализа
* Описание полученных результатов и формирование основного вывода из проведённого исследования.

###  Изучение информации.

In [ ]:
import pandas as pd #импортировал библиотеку pandas
import matplotlib.pyplot as plt #импортировал библиотеку mathplotlib
import seaborn as sns #импортировал библиотеку seaborn

In [ ]:
try:
    df = pd.read_csv(r"D:\DOCS\datasets/real_estate_data.csv", sep='\t')
except FileNotFoundError:
    df = pd.read_csv('/datasets/real_estate_data.csv', sep='\t')

In [ ]:
df.head(20) # Вывел первые 20 строчек датафрейма на экран с целью изучения данных

In [ ]:
df.info(); #получил основную информацию о датафрейме

In [ ]:
df.hist(figsize=(15, 20)); # построил общую гистограмму для всех числовых столбцов таблицы

### Предобработка данных.

In [ ]:
df.isna().sum() #получил количество пропусков в каждом столбце

In [ ]:
# check
# пропущенные значения бары

def pass_value_barh(df):
    try:
        (
            (df.isna().mean()*100)
            .to_frame()
            .rename(columns = {0:'space'})
            .query('space > 0')
            .sort_values(by = 'space', ascending = True)
            .plot(kind = 'barh', figsize = (19,6), rot = -5, legend = False, fontsize = 16)
            .set_title('Пример' + "\n", fontsize = 22, color = 'SteelBlue')    
        );    
    except:
        print('пропусков не осталось :) ')

In [ ]:
pass_value_barh(df)

In [ ]:
# заполнил пропуски в столбце balcony значением 0
df['balcony'] = df['balcony'].fillna(value=0) 

In [ ]:
# удалил строки с пропущенными значениями этажности здания, их количество незначительное, 
# восстановить эти пропуски не представляется возможным
df.dropna(subset=['floors_total'], inplace=True)

In [ ]:
# заполнил пропуски в столбце is_apartment на основании столбцов studio и open_plan, в которых пропусков не было
df['is_apartment'] = True
df.loc[df['studio'] == True, 'is_apartment'] = False
df.loc[df['open_plan'] == True, 'is_apartment'] = False

#### Работа c выбросами и пропусками в колонке  ceiling_height

In [ ]:
# Вывел описание значение в столбце ceiling_height для последующей оценки после обработки.
df['ceiling_height'].describe()

In [ ]:
# Количество пропусков в колонке ceiling_height
df['ceiling_height'].isna().sum()

In [ ]:
# Оцениваю стандартное отклонение значений высоты потолков до обработки данных
df['ceiling_height'].std()

In [ ]:
df['ceiling_height'].unique()

In [ ]:
# check
df.info()

In [ ]:
# создадим список с индексами ceiling_height от 20 до 100 с целью корректировки значений. Вероятно, при вводе данных 
# пользователь ошибочно ввел 20 вместо 2.0
ceiling_height_index = df.query('ceiling_height >= 20 & ceiling_height <= 100').index 
df.loc[ceiling_height_index,'ceiling_height'] = df.loc[ceiling_height_index,'ceiling_height'] / 10
# Удалил строки с значением высоты потолков более 10 и менее 2.5, сохраняя пропуски
df = df.loc[(df['ceiling_height'].isna()) | ((df['ceiling_height'] <= 10) & (df['ceiling_height'] >= 2.5))]
# Заполнил пропуски медианным значением
df['ceiling_height'] = df['ceiling_height'].fillna(df['ceiling_height'].median())

In [ ]:
# check
df.info()

In [ ]:
# Check
df['ceiling_height'].describe()

In [ ]:
# Check
df['ceiling_height'].std()

In [ ]:
df['ceiling_height'].isna().sum()

#### Работа c выбросами и пропусками в колонке  kitchen_area

In [ ]:
# Вывел количество пропусков в столбце kitchen_area.
df['kitchen_area'].isna().sum()

In [ ]:
# Вывел описание значение в столбце kitchen_area для последующей оценки после обработки.
df['kitchen_area'].describe()

In [ ]:
# Вывел стандартное отклонение в столбце kitchen_area для последующей оценки после обработки.
df['kitchen_area'].std()

In [ ]:
# Преобразовал значения в столбце kitchen_area от 70 до 100. И перобразовал их до 7-12 метров
kitchen_area_index = df.query('kitchen_area >= 70 & kitchen_area <= 100').index 
df.loc[kitchen_area_index,'kitchen_area'] = df.loc[kitchen_area_index,'kitchen_area'] / 10
# Удалил строки со значением менее 4 метров, сохраняя пропуски
df = df.loc[(df['kitchen_area'].isna()) | (df['kitchen_area'] >= 4)]
# Заполнил пропуски медианным значением
df['kitchen_area'] = df['kitchen_area'].fillna(df['kitchen_area'].median())

In [ ]:
# Check
df['kitchen_area'].describe()

In [ ]:
# Check
df['kitchen_area'].std()

In [ ]:
# Check
df['kitchen_area'].isna().sum()

#### Работа c выбросами и пропусками в колонке  living_area

In [ ]:
# Вывел количество пропусков в столбце living_area.
df['living_area'].isna().sum()

In [ ]:
# Вывел описание значение в столбце living_area для последующей оценки после обработки.
df['living_area'].describe()

In [ ]:
# Вывел стандартное отклонение в столбце kitchen_area для последующей оценки после обработки.
df['living_area'].std()

In [ ]:
# Преобразовал значения в столбце living_area более 200. Поделил на 10
living_area_index = df.query('living_area >= 200').index 
df.loc[living_area_index,'living_area'] = df.loc[living_area_index,'living_area'] / 10
# Удалил строки со значением менее 10 метров
df = df.loc[(df['living_area'].isna()) | (df['living_area'] >= 10)]
# Заполнил пропуски медианным значением
df['living_area'] = df['living_area'].fillna(df['living_area'].median())

In [ ]:
# Check
df['living_area'].describe()

In [ ]:
# Check
df['living_area'].std()

In [ ]:
# Check
df['living_area'].isna().sum()

In [ ]:
# Удалил строки не подъходяцие под условие
df = df.loc[(df['total_area'] * 0.9) >= (df['living_area'] + df['kitchen_area'])]

In [ ]:
# заполнил пропуски в столбцах parks_around3000 и pounds_around3000 значением 0. Так как, вероято, если люди размещавшие 
# объявление эти параметры пропустили, то их там нет
df['parks_around3000'] = df['parks_around3000'].fillna(value=0).round().astype('Int64')
df['ponds_around3000'] = df['ponds_around3000'].fillna(value=0).round().astype('Int64')

In [ ]:
# привел к int значения floors_total, last_price, balcony, floor
columns_list = ['floors_total','last_price','balcony','floor']

for column in columns_list:
    df[column] = df[column].astype('int')

In [ ]:
df.info()

In [ ]:
# преобразовал значения дня публикации объявления
df['first_day_exposition'] = pd.to_datetime(df['first_day_exposition'],format='%Y.%m.%d')

In [ ]:
# удалил строки с объявлениеями, со дня публикации которых прошло более трех лет
df.loc[(df['days_exposition'].isna()) |(df['days_exposition'] < 365*3)]
# ипривел полученные значения к целочисленному показателю
df['days_exposition'] = df['days_exposition'].round().astype('Int64')

In [ ]:
# check
data = df.copy()

In [ ]:
# check
df.info()

In [ ]:
# Удалил объявления с количеством комнат более 10
df = df.loc[(df['rooms'] <= 10)]

In [ ]:
# Удалил обхъявления с общей площадью более 250
df = df.loc[(df['total_area'] <= 250)]

In [ ]:
# Удалил объявления с датой размещения более 3 лет, сохраняя пропуски
df = df.loc[(df['days_exposition'].isna()) | (df['days_exposition'] >= 3)]

In [ ]:
# Удалил объявления с ценой менее 700000
df = df.loc[(df['last_price'] >= 700000)]

In [ ]:
df['kitchen_area'].hist(bins=50, range=(2,10))
plt.xlabel('Площадь кухни')
plt.ylabel('Количество объявлений')
plt.title('Оценка площади кухни в представленных объявлениях');

In [ ]:
# check

# Показатели о кол-ве объявлений в датасете, минимальных и максимальных значениях 
# в выбранных параметрах о продаже квартир

(
    df[['rooms', 'total_area', 'ceiling_height', 'days_exposition', 'last_price', 'living_area',  'kitchen_area',
          'floor', 'floors_total']]
    .apply (['count', 'min', 'max'])   
    .style.format("{:,.2f}")
)

In [ ]:
# check
data.hist(column = 'days_exposition', bins = 50, figsize = (15,3), range = (0,5));

data.hist(column = 'days_exposition', bins = 50, figsize = (15,3), range = (0,5))
plt.ylim(0, 40);

In [ ]:
# check
df.rooms.value_counts().to_frame()

In [ ]:
# check
data.total_area.hist(bins = 150, figsize = (15,3), range=(0,250));

In [ ]:
# check
df.total_area.hist(bins = 150, figsize = (15,3), range = (180,250));

In [ ]:
# check
df.total_area.hist(bins = 15, figsize = (15,3), range = (180,250));

In [ ]:
# check

# Значения параметров объектов недвижимости на разных квантилях

(
    data[['rooms', 'total_area', 'ceiling_height', 'days_exposition', 'last_price', 'living_area',  
        'kitchen_area', 'floor',   'floors_total']]
    .quantile([0.0012, 0.01, .5, .99, .9988]) # выбираем размах в 0,9976 квантилей 
    .style.format("{:,.2f}")
)

### Заполнены пропущенные значения в столбцах balcony. ceiling_height. total_floors. is_apartment. Данные пропуски, вероятно, возникли в результате человесческого фактора. То есть, не были внесены пользователем, размещающим объявление.

In [ ]:
df['locality_name'] = df['locality_name'].str.lower() #привел все значения в locality_name к нижнему регистру
df['locality_name'] = df['locality_name'].str.replace('ё','е') # заменил все ё на е
# создал список, который мог ктороый поможет избежать появления неявных дубликатов 
locality_name_list = ['городской поселок',
                      'деревня', 
                      'поселок городского типа',
                      'поселок станции',
                      'садоведческое некоммерческое товарищество',
                      'садовое товарищество',
                      'село',
                      'коттеджный поселок',
                      'городской поселок',
                      'пок']
def locality_name_replace(data, column, locality_name_list): # создал функцию, для замены значений localyti_name_list 
    for locality_name in locality_name_list:                 #в столбце 'locality_name' на пустоту
        data[column] = data[column].str.replace(locality_name, '')
        
locality_name_replace(df,'locality_name', locality_name_list)
df['locality_name'].str.strip().unique() # удалил пробелы перед названиями населенных пунктов

In [ ]:
# df.dropna(subset='locality_name', inplace=True) # удалил строки с пропущенными значениями locality_name

In [ ]:
# check
df['locality_name'].nunique()

In [ ]:
# check
data = df.copy()

In [ ]:
# check
data.info()

In [ ]:
# check

# Показатели о кол-ве объявлений в датасете, минимальных и максимальных значениях 
# в выбранных параметрах о продаже квартир

(
    df[['rooms', 'total_area', 'ceiling_height', 'days_exposition', 'last_price', 'living_area',  'kitchen_area',
          'floor', 'floors_total']]
    .apply (['count', 'min', 'max'])   
    .style.format("{:,.2f}")
)

In [ ]:
# check
data.rooms.value_counts().to_frame()

In [ ]:
# check
data.total_area.hist(bins = 150, figsize = (15,3));

In [ ]:
# check
data.total_area.hist(bins = 150, figsize = (15,3), range = (25,250));

In [ ]:
# check
data.total_area.hist(bins = 15, figsize = (15,3), range = (25,250));

In [ ]:
# check

# Значения параметров объектов недвижимости на разных квантилях

(
    data[['rooms', 'total_area', 'ceiling_height', 'days_exposition', 'last_price', 'living_area',  
        'kitchen_area', 'floor',   'floors_total']]
    .quantile([0.0012, 0.01, .5, .99, .9988]) # выбираем размах в 0,9976 квантилей 
    .style.format("{:,.2f}")
)

### Добавление в таблицу новых столбцов.

In [ ]:
# добавил столбец с ценой одного квадратного метра
df['price_of_m2'] = df['last_price'] / df['total_area']

In [ ]:
# добавил столбец c типом этажа квартиры (значения — «первый», «последний», «другой»).
def categorize_floors(df):
    if df['floor'] == 1:
        return 'первый'
    elif df['floor'] < df['floors_total']:
        return 'другой'
    elif df['floor'] == df['floors_total']:
        return 'последний'
    else:
        return 'не найдено'

df['floor_category'] = df.apply(categorize_floors, axis=1)

In [ ]:
df['floor'].unique()
# Еулевых и отрицательных значений не выявлено

In [ ]:
# добавил столбец с днем недели публикации объявления.
df['publication_day'] = df['first_day_exposition'].dt.dayofweek

In [ ]:
# добавил столбец с месяцем публикации объявления.
df['publication_month'] = df['first_day_exposition'].dt.month

In [ ]:
# добавил столбец с годом недели публикации объявления.
df['publication_year'] = df['first_day_exposition'].dt.year

In [ ]:
# добавил столбец с расстоянием до центра города в километрах.
df['cityCenters_nearest_km'] = round(df['cityCenters_nearest'] / 1000)
try:
    df['cityCenters_nearest_km'] = df['cityCenters_nearest_km'].astype('int')
except:
    pass

### Исследовательский анализ данных.

In [ ]:
# Постороил гистограмму описывающую общую площадь квартир.
df.hist('total_area',bins=100, range=(20,250), figsize=(15, 5));

#### По гистограмме можно судить о том что подавляющее число кваритр, размещенных в объявляниях имеют площадь менее 100 метров в квадрате

In [ ]:
# Постороил гистограмму описывающую жилую площадь квартир. Удалил строки с жилой площадью менее 10 метров
df= df[(df['living_area'] >= 10)]
df.hist('living_area', bins=100, range=(10,100), figsize=(15, 5));

#### По данной гистограмме можно сказать, что бОльшая часть квартир имеют жилую клощадь от 10 до 60 метров в квадрате, притом пик приходится на значения до 30 метров в квадрате.

In [ ]:
# Постороил гистограмму описывающую площадь кухни.
df.hist('kitchen_area', bins=27, range=(3,50), figsize=(15, 5));

#### По данной гистограмме можно сказать, что бОльшая часть квартир имеет площадь кухни не превышающаю 17 метров в квадрате. Пики приходятся на значения до 10 метров в квадрате.

In [ ]:
# Постороил гистограмму описывающую полную стоимость квартир;
df = df[(df['last_price'] <= 600000000)]
df.hist('last_price', bins=100, range=(500000,20000000), figsize=(15, 5));

#### По данной гистограмме можно сказать, что бОльшая часть квартир имеет цену до 10 миллионов. Пик цен приходится до отметки в пять миллионов

In [ ]:
# Вывел уникальные значения количества комнат;
df['rooms'].sort_values().unique()
# Отсеял квартиры с количеством комнат больше 10
df.hist('rooms', bins=20, range=(0,10));
df['rooms'].describe()

In [ ]:
# Постороил гистограмму, описывающую высоту потолков в квартирах. Отсеял квартиры с высотой потолков менее 2.5 и более 10
df.hist('ceiling_height', bins=6, range=(2,5));

#### По данной гистограмме можно сказать, что в подавляющем большинстве квартир потолки не выше 3.5 метров

In [ ]:
# Постороил гистограмму, описывающую этаж квартиры;
df.hist('floor', bins=33);

#### По данной гистограмме можно сказать, что большинство квартир расоложено на этажах ниже десятого.

In [ ]:
# Постороил гистограмму, описывающую тип этажа квартиры (первый, последний, другой)
df['floor_category'].hist(bins=6);

#### По данной гистограмме можно сказать, что сумма квартир на первом и последнем этажах мешьше, чем на других.

In [ ]:
# Постороил гистограмму, описывающую общее количество этажей в доме
df['floors_total'].sort_values().unique()
df.hist('floors_total', bins=60, range=(0,30));

#### По данной гистограмме можно сказать, что среди зданий, квартиры в которых представлены на продажу, преобладают пятиэтажные дома.

In [ ]:
#Постороил гистограмму, описывающую расстояние до центра города в метрах
df['cityCenters_nearest'].sort_values().unique()
df.hist('cityCenters_nearest',bins=60, range=(100,66000));

#### По данной гистограмме можно сказать, что большинство квартир расположено в 20000 метрах или в 20 километрах от центра города.

In [ ]:
# Постороил гистограмму, описывающую расстояние до ближайшего аэропорта
df['airports_nearest'].sort_values().unique()
df = df[(df['airports_nearest'] >= 5000)]
df.hist('airports_nearest', bins=85);

#### По данной гистограмме можно сделать вывод, что большинство квартир расположено на расстоянии выше 10 километров от аэропорта.

In [ ]:
# Постороил гистограмму, описывающую расстояние ближайшего парка
df.hist('parks_nearest', bins=100, range=(50,3200));

#### По данной гистограмме можно сделать вывод, что в большинстве случаев ближайший парк расположен на расстоянии до одного километра от квартиры.

In [ ]:
# Построил гистограмму, описывающую в какой день недели было размещено объявление.
df.hist('publication_day', bins=14);

#### По данной гстограмме можно сделать вывод, что минимальное количество объявлений размещается по выходным дням.

In [ ]:
# Построил гистограмму, описывающую в каком месяце было размещено объявление.
df.hist('publication_month', bins=15);

#### По данной гистограмме можно сделать вывод, что пик размещения объявлений приходится на февраль. Идет на спад до марта. Минимальное количество размещенных объявлений в мае. Затем с сентября до ноября отмечается тенденция к росту, резко спадающая в декабре. Январь также непопулярен, с точки зрения размещения объявлений.

#### Февраль опережает остальные, вероятно, потому, что за 2019 год есть данные только за начало года. Или это может быть связано с человеческим фактором, как и маленькое количество подачи объявлений в мае.

In [ ]:
#Построил гистограмму, описывающую в каком году было размещено объявление.
df.hist('publication_year', bins=12);

#### Минимальное количество размещенных объявлений отмечается в 2014 году. Максимальное в 2017, но в дальнейшем отмечается спад. Интересно, связан ли спад с измененией работы платформы или с изменениями тенденций на рынке.

In [ ]:
# Изучите, как быстро продавались квартиры
df.hist('days_exposition', bins=100, range=(1,1094))
print('Среднее значение дней экспозиции:', df['days_exposition'].mean())
print('Медианное значение дней экспозиции:', df['days_exposition'].median())

In [ ]:
df.boxplot(column='days_exposition')
plt.ylim(0,600)
plt.show()

#### По данной гистограмме можно судить о том, что большинство квартир было продано менее чем за 200 дней, меианное значение 120 дней, думаю, что к быстрым продажам можно отнести те, что были сделаны меньше, чем за 120 дней. В свою очередь те, что были сделаны позже 200 дней можно считать долгими.

#### К быстрым продажам можно отнести продажи менее Q1 (0-30 дней),  к длительным продажам можно отнести продажи выше Q3 (от 220 дней)

In [ ]:
# Постороил диаграмму рассеивания зависимости цены от общей площади квартиры.
df.plot(x='last_price', y='total_area', kind='scatter', sharex=False, grid=True, alpha=0.3);

#### По данной диаграмме рассеивания мы видим положительную корреляцию. С увеличением площади, растет цена.   В данной выборке предложений можно найти как небольшую, так и большую квартиру за схожую стоимость. Могу предположить, что связано это с местом расположения. Также мы видим отдельные точки, которые, по всей видимости, явсляются выбросами.

In [ ]:
# Постороил диаграмму рассеивания зависимости цены от жилой площади квартиры.
df.plot(x='last_price', y='living_area', kind='scatter', sharex=False, grid=True, alpha=0.3);
correlation=df['last_price'].corr(df['living_area'])
print(f'Корреляция цены и жилой площади составяляет:{correlation:0.2%}')

#### В данной диаграмме мы не видим сильных отличий от предыдущей. Мы видим положительную корреляцию. 

In [ ]:
# Постороил диаграмму рассеивания зависимости цены от площади кухни в квартире.
df.plot(x='last_price', y='kitchen_area', kind='scatter', sharex=False, grid=True, alpha=0.3);
correlation = df['last_price'].corr(df['kitchen_area'])
print(f'Корреляция цены и площади кухни составляет:{correlation:0.2%}')

#### В данной диаграмме мы видим положительную корреляцию.  Однако, в зависимость не столь очевидна, как в связке цена-жилая площадь.

In [ ]:
# Постороил диаграмму рассеивания зависимости цены от количества комнат в квартире.
df.plot(x='last_price', y='rooms', kind='scatter', sharex=False, grid=True, alpha=0.3);
correlation = df['last_price'].corr(df['rooms'])
print(f'Корреляция цены и количества комнат составляет:{correlation:0.2%}')

#### В данной диаграмме нельзя назвать корреляцию положительной. Прямой зависимоси цены от количества комнат не выявлено. Мы видим, что наибольши диапозон цен в квартирах от 4 до 6 комнат. В то время, как диапозон однокомнатных и 10 комнатных является одинаковым.

In [ ]:
# Постороил диаграмму рассеивания зависимости цены от этажа, на котором расположена квартира.
df.pivot_table(index='floor_category',values='last_price',aggfunc='mean')\
    .plot(y='last_price', kind='barh', title='mean');
df.pivot_table(index='floor_category',values='last_price',aggfunc='median')\
    .plot(y='last_price', kind='barh', title='median');

#### По данным диаграммам можно судить от том что недвижимость на последнем этаже не сильно отличается в цене от других этажей, но заметно выше чем на первом.

In [ ]:
# Постороил диаграмму рассеивания зависимости цены от дня недели размещения объявления.
df.pivot_table(index='publication_day',values='last_price',aggfunc='mean')\
    .plot(y='last_price', kind='line', title='mean');
df.pivot_table(index='publication_day',values='last_price',aggfunc='median')\
    .plot(y='last_price', kind='line', title='median');

#### По данным диаграммам мы можем судить о том, что объявления с самой догрогой недвижимостью публикуются в четверг

In [ ]:
# Постороил диаграмму рассеивания зависимости цены от месяца размещения объявления.
df.pivot_table(index='publication_month',values='last_price',aggfunc='mean')\
    .plot(y='last_price', kind='line', title='mean');
df.pivot_table(index='publication_month',values='last_price',aggfunc='median')\
    .plot(y='last_price', kind='line', title='median');

#### По данным диаграммам можно сказать, что объявления с дешевой недвижимостью можно было увидеть в июне и октябре. С самой дорогой в декабре.

In [ ]:
# Постороил диаграмму рассеивания зависимости цены от года размещения объявления.
df.pivot_table(index='publication_year',values='last_price',aggfunc='mean')\
    .plot(y='last_price', kind='line', title='mean');
df.pivot_table(index='publication_year',values='last_price',aggfunc='median')\
    .plot(y='last_price', kind='line', title='median');

#### По данным диаграммам мы видим спад цен недвиджисмости с 2014 года. Вероятно это связано с кризисом на рынке, падением курса рубля, сответственно, снижению цен на недвижимость.

In [ ]:
# Построил таюлицу где объединил столюцы сумм и количества значений цен и общей площади.
real_estate = df.pivot_table(index='locality_name', values=['last_price','total_area'],aggfunc=['sum','count'])
# Переименовал столбцы в созданной таблице
real_estate.columns = ['сумма цен', 'сумма значений','количество цен','количество значений']
# Результат
real_estate

In [ ]:
# Отсортировал полученную таблицу по количеству объявлений
real_estate = real_estate.sort_values(by='количество значений', ascending=False)
# Результат
real_estate

In [ ]:
# Для дальнейшего анализа выбрал первые 10 отсортированных ранее строк
real_estate = real_estate.iloc[:10]
# Результат
real_estate

In [ ]:
# Добавил столбец со средней ценой квадратного метра для этого общую стоимость всех объявлений разделил 
# на количество всех объявлений
real_estate['стоимость м2'] = real_estate['сумма цен']/real_estate['сумма значений']
real_estate.style.format({'стоимость м2': '₽{:.2f}'})

In [ ]:
# Отсортировал таблицу по столбцу цены одного квадратного метра 
real_estate.sort_values(by=['стоимость м2'],ascending=True, inplace=True)
real_estate['стоимость м2'].plot(kind='barh');

#### По таблице мы можем сказать, что самая высокая стоимость квадратного метра в Санкт-Петербурге, а самая низкая в Красном селе.  

In [ ]:
# Сделал срез данных по городу Санкт-Петербург, на основании которого построил таблицу и диаграмму
(
    df[df['locality_name'] == 'санкт-петербург']
        .pivot_table(index='cityCenters_nearest_km', values='last_price', aggfunc='mean')
        .plot(kind='bar')
);

#### Самая большая цена у недвижимости с расстоянием до 1 километра от центра, далее цена снижается с увеличением расстояния от 1 до 3 километров. Далее мы видим рост от  4 до 7 километровю Затем цена падает с увеличением расстояния от 8 до 26 километров. Всплеск цен на расстоянии 27 километров связан с выявлением на этом расстоянии от центра одного объявления о продоже с высокой стоимостью.

In [ ]:
# отдельно исследовал возростание цены на 27 км
df[(df['cityCenters_nearest_km'] == 27) & (df['locality_name'] == 'санкт-петербург')]

### Вывод

#### Наибольшее влияние на цену квартиры оказывает общий размер квартиры, количество комнат, в большей степени размер жилой площади, в меньшей- кухни. 
#### Квартиры на первом этаже уступают в цене квартирам на последующих.
#### Наибольшую цену за квадратный метр имеет недвижимость расмолоенная в Санкт-Петербурге, либо в области, но близко к Санкт-Петербургу. В самом же Санкт-Петербурге  есть как дешевая, так и дорогая недвижимость в разной удаленности от центра города, наибольшую же цену имеет недвижимость в центре.